In [33]:
import pandas as pd

df_orders = pd.read_csv('orders.csv')

print("Orders Data Loaded:")
print(df_orders.head())

Orders Data Loaded:
   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  


In [34]:
df_users = pd.read_json('users.json')

print("Users Data Loaded:")
print(df_users.head())

Users Data Loaded:
   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold


In [35]:
import sqlite3

with open('restaurants.sql', 'r') as f:
    sql_script = f.read()

conn = sqlite3.connect(':memory:')

cursor = conn.cursor()
cursor.executescript(sql_script)
conn.commit()

df_restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)

print("Restaurants Data Loaded:")
print(df_restaurants.head())

Restaurants Data Loaded:
   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [36]:
merged_df = pd.merge(df_orders, df_users, on='user_id', how='left')

final_df = pd.merge(
    merged_df,
    df_restaurants,
    on='restaurant_id',
    how='left',
    suffixes=('', '_from_restaurants')
)

if 'restaurant_name_from_restaurants' in final_df.columns:
    final_df.drop(columns=['restaurant_name_from_restaurants'], inplace=True)

print(final_df.columns.tolist())
print(final_df.shape)
final_df.head()

output_filename = 'final_food_delivery_dataset.csv'
final_df.to_csv(output_filename, index=False)

print(f"\nDataset successfully saved to {output_filename}")

files.download(output_filename)

['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name', 'name', 'city', 'membership', 'cuisine', 'rating']
(10000, 11)

Dataset successfully saved to final_food_delivery_dataset.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
gold_revenue_df = final_df[final_df['membership'] == 'Gold']

city_gold_revenue = gold_revenue_df.groupby('city')['total_amount'].sum()

city_gold_revenue_sorted = city_gold_revenue.sort_values(ascending=False)

print("Total Revenue by City (Gold Members):")
print(city_gold_revenue_sorted)

top_city = city_gold_revenue_sorted.idxmax()
highest_revenue = city_gold_revenue_sorted.max()

print(f"City with the highest revenue from Gold members: {top_city}")
print(f"Total Revenue: {highest_revenue:.2f}")

Total Revenue by City (Gold Members):
city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64
City with the highest revenue from Gold members: Chennai
Total Revenue: 1080909.79


In [41]:
cuisine_aov = final_df.groupby('cuisine')['total_amount'].mean()

cuisine_aov_sorted = cuisine_aov.sort_values(ascending=False)

print("Average Order Value by Cuisine:")
print(cuisine_aov_sorted)

top_cuisine = cuisine_aov_sorted.idxmax()
highest_average = cuisine_aov_sorted.max()

print(f"Cuisine with the highest average order value: {top_cuisine}")
print(f"Average Order Value: {highest_average:.2f}")

Average Order Value by Cuisine:
cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64
Cuisine with the highest average order value: Mexican
Average Order Value: 808.02


In [42]:
user_spending = final_df.groupby('user_id')['total_amount'].sum()

high_value_users = user_spending[user_spending > 1000]

count = len(high_value_users)

print(f"Number of distinct users with total orders > 1000: {count}")

print("Details of users with spending > 1000:")
print(high_value_users.sort_values(ascending=False))

Number of distinct users with total orders > 1000: 2544
Details of users with spending > 1000:
user_id
1515    11556.49
650     10747.44
496      9634.30
2586     9486.61
2615     9237.32
          ...   
1814     1013.08
686      1011.84
192      1010.38
625      1005.21
346      1001.62
Name: total_amount, Length: 2544, dtype: float64


In [43]:
clean_df = final_df.dropna(subset=['rating'])

bins = [3.0, 3.5, 4.0, 4.5, 5.1]
labels = ['3.0 - 3.5', '3.5 - 4.0', '4.0 - 4.5', '4.5 - 5.0']

clean_df['rating_range'] = pd.cut(clean_df['rating'], bins=bins, labels=labels)

revenue_by_range = clean_df.groupby('rating_range')['total_amount'].sum()

revenue_by_range_sorted = revenue_by_range.sort_values(ascending=False)

print(revenue_by_range_sorted)

top_range = revenue_by_range_sorted.idxmax()
highest_revenue = revenue_by_range_sorted.max()

print(f"Rating range with highest revenue: {top_range}")
print(f"Total Revenue: {highest_revenue:.2f}")

rating_range
4.5 - 5.0    2197030.75
4.0 - 4.5    1960326.26
3.0 - 3.5    1881754.57
3.5 - 4.0    1717494.41
Name: total_amount, dtype: float64
Rating range with highest revenue: 4.5 - 5.0
Total Revenue: 2197030.75


/tmp/ipython-input-1168243234.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  revenue_by_range = clean_df.groupby('rating_range')['total_amount'].sum()


In [44]:
gold_data = final_df[final_df['membership'] == 'Gold']

city_avg_order_value = gold_data.groupby('city')['total_amount'].mean()

city_avg_order_value_sorted = city_avg_order_value.sort_values(ascending=False)

print(city_avg_order_value_sorted)

top_city = city_avg_order_value_sorted.idxmax()
highest_avg = city_avg_order_value_sorted.max()

print(f"City with the highest average order value (Gold Members): {top_city}")
print(f"Average Order Value: {highest_avg:.2f}")

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64
City with the highest average order value (Gold Members): Chennai
Average Order Value: 808.46


In [45]:
cuisine_stats = final_df.groupby('cuisine').agg(
    distinct_restaurant_count=('restaurant_id', 'nunique'),
    total_revenue=('total_amount', 'sum')
).reset_index()

cuisine_stats_sorted = cuisine_stats.sort_values('distinct_restaurant_count', ascending=True)

print(cuisine_stats_sorted)

min_count = cuisine_stats_sorted['distinct_restaurant_count'].min()

potential_candidates = cuisine_stats_sorted[cuisine_stats_sorted['distinct_restaurant_count'] == min_count]

target_cuisine = potential_candidates.sort_values('total_revenue', ascending=False).iloc[0]

print(f"Cuisine with the lowest number of distinct restaurants ({min_count} restaurants):")
print(f"Target Cuisine: {target_cuisine['cuisine']}")
print(f"Total Revenue: {target_cuisine['total_revenue']:.2f}")

target_revenue_per_rest = target_cuisine['total_revenue'] / target_cuisine['distinct_restaurant_count']
print(f"Revenue per Restaurant: {target_revenue_per_rest:.2f}")

   cuisine  distinct_restaurant_count  total_revenue
0  Chinese                        120     1930504.65
1   Indian                        126     1971412.58
2  Italian                        126     2024203.80
3  Mexican                        128     2085503.09
Cuisine with the lowest number of distinct restaurants (120 restaurants):
Target Cuisine: Chinese
Total Revenue: 1930504.65
Revenue per Restaurant: 16087.54


In [46]:
total_orders = len(final_df)

gold_orders = len(final_df[final_df['membership'] == 'Gold'])

percentage = (gold_orders / total_orders) * 100

percentage_rounded = round(percentage)

print(f"Total Orders: {total_orders}")
print(f"Orders by Gold Members: {gold_orders}")
print(f"\nPercentage of total orders by Gold members: {percentage_rounded}%")

Total Orders: 10000
Orders by Gold Members: 4987

Percentage of total orders by Gold members: 50%


In [47]:
restaurant_stats = final_df.groupby(['restaurant_id', 'restaurant_name']).agg(total_orders=('order_id', 'count'),total_revenue=('total_amount', 'sum')).reset_index()

low_volume_restaurants = restaurant_stats[restaurant_stats['total_orders'] < 20]

low_volume_restaurants['avg_order_value'] = low_volume_restaurants['total_revenue'] / low_volume_restaurants['total_orders']

top_restaurant = low_volume_restaurants.sort_values('avg_order_value', ascending=False).head(1)

print(f"Restaurant: {top_restaurant['restaurant_name'].values[0]}")
print(f"Total Orders: {int(top_restaurant['total_orders'].values[0])}")
print(f"Average Order Value: {top_restaurant['avg_order_value'].values[0]:.2f}")

print(low_volume_restaurants.sort_values('avg_order_value', ascending=False)[['restaurant_name', 'total_orders', 'avg_order_value']])

Restaurant: Hotel Dhaba Multicuisine
Total Orders: 13
Average Order Value: 1040.22
                 restaurant_name  total_orders  avg_order_value
293     Hotel Dhaba Multicuisine            13      1040.222308
261           Hotel Cafe Punjabi            18      1029.473333
76              Sri Mess Punjabi            12      1029.180833
192  Swagath Eatery Multicuisine            15      1026.306667
6          Ruchi Biryani Punjabi            16      1002.140625
..                           ...           ...              ...
183    Annapurna Tiffins Punjabi            19       621.828947
497       Darbar Tiffins Non-Veg            18       596.815556
191    Darbar Restaurant Punjabi            14       589.972857
328       Spice Tiffins Pure Veg            15       578.578667
299           Ruchi Mess Punjabi            17       572.686471

[241 rows x 3 columns]


/tmp/ipython-input-2673097898.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_volume_restaurants['avg_order_value'] = low_volume_restaurants['total_revenue'] / low_volume_restaurants['total_orders']


In [48]:
combo_revenue = final_df.groupby(['membership', 'cuisine'])['total_amount'].sum().reset_index()

combo_revenue_sorted = combo_revenue.sort_values('total_amount', ascending=False)

options = [
    ('Gold', 'Indian'),
    ('Gold', 'Italian'),
    ('Regular', 'Indian'),
    ('Regular', 'Chinese')
]

filtered_combos = combo_revenue_sorted[combo_revenue_sorted.set_index(['membership', 'cuisine']).index.isin(options)]

print(filtered_combos.to_string(index=False))

top_combo = filtered_combos.iloc[0]
print(f"Highest Revenue Combination: {top_combo['membership']} + {top_combo['cuisine']}")
print(f"Total Revenue: {top_combo['total_amount']:.2f}")

membership cuisine  total_amount
      Gold Italian    1005779.05
   Regular  Indian     992100.27
      Gold  Indian     979312.31
   Regular Chinese     952790.91
Highest Revenue Combination: Gold + Italian
Total Revenue: 1005779.05


In [49]:
final_df['order_date_dt'] = pd.to_datetime(final_df['order_date'], format='%d-%m-%Y')

final_df['quarter'] = final_df['order_date_dt'].dt.quarter

quarter_revenue = final_df.groupby('quarter')['total_amount'].sum()

quarter_revenue_sorted = quarter_revenue.sort_values(ascending=False)

quarter_names = {1: 'Q1 (Jan-Mar)', 2: 'Q2 (Apr-Jun)', 3: 'Q3 (Jul-Sep)', 4: 'Q4 (Oct-Dec)'}
quarter_revenue_sorted.index = quarter_revenue_sorted.index.map(quarter_names)

print(quarter_revenue_sorted)

peak_quarter = quarter_revenue_sorted.index[0]
highest_revenue = quarter_revenue_sorted.iloc[0]

print(f"Quarter with the highest total revenue: {peak_quarter}")
print(f"Total Revenue: {highest_revenue:.2f}")

quarter
Q3 (Jul-Sep)    2037385.10
Q4 (Oct-Dec)    2018263.66
Q1 (Jan-Mar)    2010626.64
Q2 (Apr-Jun)    1945348.72
Name: total_amount, dtype: float64
Quarter with the highest total revenue: Q3 (Jul-Sep)
Total Revenue: 2037385.10


In [52]:
gold_orders_df = final_df[final_df['membership'] == 'Gold']
total_gold_orders = len(gold_orders_df)
print(f"Total orders placed by Gold members: {total_gold_orders}")

Total orders placed by Gold members: 4987


In [53]:
hyderabad_orders = final_df[final_df['city'] == 'Hyderabad']
total_revenue = hyderabad_orders['total_amount'].sum()
rounded_revenue = round(total_revenue)
print(f"Total revenue from Hyderabad (rounded): {rounded_revenue}")

Total revenue from Hyderabad (rounded): 1889367


In [54]:
distinct_users_count = final_df['user_id'].nunique()
print(f"Total distinct users who placed at least one order: {distinct_users_count}")

Total distinct users who placed at least one order: 2883


In [55]:
gold_data = final_df[final_df['membership'] == 'Gold']
avg_order_value = gold_data['total_amount'].mean()
rounded_avg = round(avg_order_value, 2)
print(f"Average order value for Gold members: {rounded_avg}")

Average order value for Gold members: 797.15


In [56]:
high_rated_orders = final_df[final_df['rating'] >= 4.5]
order_count = len(high_rated_orders)
print(f"Total orders in restaurants with rating >= 4.5: {order_count}")

Total orders in restaurants with rating >= 4.5: 3374


In [57]:
gold_df = final_df[final_df['membership'] == 'Gold']
city_revenue_gold = gold_df.groupby('city')['total_amount'].sum()
top_city = city_revenue_gold.idxmax()
orders_in_top_city = len(gold_df[gold_df['city'] == top_city])

print(f"Top revenue city among Gold members: {top_city}")
print(f"Total orders in that city (Gold only): {orders_in_top_city}")

Top revenue city among Gold members: Chennai
Total orders in that city (Gold only): 1337
